# jobs

> Fill in a module description here

In [ ]:
#| default_exp jobs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
from apscheduler.triggers.cron import CronTrigger
from apscheduler.jobstores.memory import MemoryJobStore
from apscheduler.executors.pool import ThreadPoolExecutor, ProcessPoolExecutor
from apscheduler.events import EVENT_JOB_EXECUTED, EVENT_JOB_ERROR
from triggerkit import util
import time
import logging
import os
import json
from triggerkit.shared_actions import SharedContext
from triggerkit.snowflake import get_view_data
from triggerkit.actions import run, register
from datetime import datetime, time as datetime_time
from typing import Dict, List, Callable, Optional, Any, Union, Tuple

2025-05-21 14:21:14,964 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2025-05-21 14:21:15,329 - triggerkit - INFO - Registered action: Register Views
2025-05-21 14:21:15,330 - triggerkit - INFO - Registered action: Get DDL and Register Views


In [ ]:
#| export

# Initialize the scheduler with both thread and process pool executors
executors = {
    'default': ThreadPoolExecutor(20),  # Default thread pool for I/O-bound jobs
    'processpool': ProcessPoolExecutor(5)  # Process pool for CPU-bound jobs
}

job_defaults = {
    'coalesce': True,  # Combine missed jobs into a single job execution
    'max_instances': 3,  # Maximum concurrent instances of a job
    'misfire_grace_time': 60  # Seconds for which misfired jobs are still executed
}

# Initialize the scheduler
scheduler = BackgroundScheduler(
    executors=executors,
    job_defaults=job_defaults,
    timezone='UTC'  # Use UTC for consistent scheduling across timezones
)
scheduler.add_jobstore(MemoryJobStore(), 'default')

In [ ]:
#| export

def create(view_name: str, 
           action_names: Union[str, List[str]], 
           job_name: Optional[str] = None, 
           executor: str = 'default'):
    """
    Create a job function that fetches data from a view and runs specified actions.
    
    Args:
        view_name: Name of the registered view
        action_names: Name or list of names of registered actions
        job_name: Optional name for the job
        executor: Name of the executor to use ('default' for thread pool, 'processpool' for process pool)
        
    Returns:
        Job function
    """
    # TODO: Add print statements
    if isinstance(action_names, str):
        action_names = [action_names]
    
    if not job_name:
        job_name = f"{view_name}_{'_'.join(action_names)}_job"
    
    def job():
        start_time = datetime.now()
        util.logger.info(f"Running job '{job_name}' ...")
        try:
            # Fetch data from view
            data = get_view_data(view_name)
            
            # Run each action
            results = {}
            context = SharedContext()
            for action_name in action_names:
                results[action_name] = run(action_name, data, context)
                context.clear_updates()
                util.logger.debug(f"[context] '{action_name}' executed successfully.")
            
            execution_time = datetime.now() - start_time
            util.logger.info(f"Job '{job_name}' completed successfully in {execution_time.total_seconds():.2f}s")
            return results
        except Exception as e:
            execution_time = datetime.now() - start_time
            util.logger.error(f"Job '{job_name}' failed after {execution_time.total_seconds():.2f}s: {str(e)}")
            raise
    
    job.__name__ = job_name
    return job

In [ ]:
#| export
util.SCHEDULED_JOBS = {}

def _job_listener(event):
    """Event listener for job execution."""
    if event.exception:
        util.logger.error(f"Job '{event.job_id}' raised an exception: {event.exception}")
    else:
        util.logger.debug(f"Job '{event.job_id}' executed successfully with result: {event.retval}")

# Add listeners to the scheduler
scheduler.add_listener(_job_listener, EVENT_JOB_EXECUTED | EVENT_JOB_ERROR)

def _parse_cron_expression(cron_expr: str) -> Dict[str, Any]:
    """
    Parse a cron expression into parameters for CronTrigger.
    
    Args:
        cron_expr: Cron expression (e.g., "* * * * *" or "0 8 * * *")
        
    Returns:
        Dictionary of parameters for CronTrigger
    """
    parts = cron_expr.strip().split()
    
    # Standard cron has 5 parts: minute, hour, day of month, month, day of week
    if len(parts) != 5:
        raise ValueError(f"Invalid cron expression: {cron_expr}. Expected 5 fields.")
    
    cron_params = {}
    
    # Parse minute (0-59)
    if parts[0] != '*':
        cron_params['minute'] = parts[0]
    
    # Parse hour (0-23)
    if parts[1] != '*':
        cron_params['hour'] = parts[1]
    
    # Parse day of month (1-31)
    if parts[2] != '*':
        cron_params['day'] = parts[2]
    
    # Parse month (1-12)
    if parts[3] != '*':
        cron_params['month'] = parts[3]
    
    # Parse day of week (0-6 or MON-SUN)
    if parts[4] != '*':
        cron_params['day_of_week'] = parts[4]
    
    return cron_params

def _create_trigger_from_config(job_config: Dict[str, Any]) -> Tuple[Any, str]:
    """
    Creates an appropriate trigger based on job configuration.
    
    Args:
        job_config: Job configuration dictionary
        
    Returns:
        Tuple of (trigger object, schedule description)
    """
    schedule_str = job_config.get('schedule')
    run_at = job_config.get('run_at')
    cron_schedule = job_config.get('cron_schedule')
    
    # Check for cron schedule first
    if cron_schedule:
        try:
            cron_params = _parse_cron_expression(cron_schedule)
            trigger = CronTrigger(**cron_params)
            return trigger, f"cron: {cron_schedule}"
        except ValueError as e:
            util.logger.warning(f"Invalid cron expression: {str(e)}")
            # Fall back to interval scheduling if available
    
    # Process interval schedule
    if schedule_str:
        schedule_parts = schedule_str.split()
        if len(schedule_parts) >= 3 and schedule_parts[0] == "every":
            try:
                interval = int(schedule_parts[1])
                unit = schedule_parts[2].lower()
                
                # Configure trigger based on schedule
                if unit in ('minute', 'minutes'):
                    trigger = IntervalTrigger(minutes=interval)
                    return trigger, f"every {interval} {unit}"
                elif unit in ('hour', 'hours'):
                    trigger = IntervalTrigger(hours=interval)
                    return trigger, f"every {interval} {unit}"
                elif unit in ('day', 'days'):
                    # If run_at is specified, use CronTrigger for daily jobs
                    if run_at:
                        try:
                            hour, minute = map(int, run_at.split(':'))
                            # Create cron trigger to run at specific time
                            trigger = CronTrigger(hour=hour, minute=minute)
                            return trigger, f"daily at {run_at}"
                        except (ValueError, AttributeError):
                            util.logger.warning(f"Invalid run_at time format: {run_at}, expected HH:MM. Using interval trigger.")
                            trigger = IntervalTrigger(days=interval)
                            return trigger, f"every {interval} {unit}"
                    else:
                        trigger = IntervalTrigger(days=interval)
                        return trigger, f"every {interval} {unit}"
                elif unit in ('week', 'weeks'):
                    trigger = IntervalTrigger(weeks=interval)
                    return trigger, f"every {interval} {unit}"
                else:
                    raise ValueError(f"Unsupported schedule unit: {unit}")
            except ValueError as e:
                raise ValueError(f"Invalid schedule format: {schedule_str} - {str(e)}")
        else:
            raise ValueError(f"Unsupported schedule format: {schedule_str}")
    
    raise ValueError("No valid scheduling information provided (cron_schedule or schedule)")

In [ ]:
#| export

def schedule_jobs(config: Dict[str, Any]):
    """
    Schedule jobs based on TOML configuration.
    
    Args:
        config: Dictionary containing configuration sections
        
    Returns:
        None
    """
    jobs = config.get('jobs', [])
    
    for job_config in jobs:
        name = job_config.get('name')
        view_name = job_config.get('view')
        actions = job_config.get('actions', [])
        enabled = job_config.get('enabled', True)
        executor = job_config.get('executor', 'default')
        
        if not enabled:
            util.logger.info(f"Job '{name}' is disabled, skipping")
            continue
            
        if not all([view_name, actions]) or not (job_config.get('schedule') or job_config.get('cron_schedule')):
            util.logger.warning(f"Skipping invalid job configuration: {job_config}")
            continue
        
        job_func = create(view_name, actions, name, executor)
        
        try:
            # Create trigger from configuration
            trigger, schedule_description = _create_trigger_from_config(job_config)
            
            # Add job to scheduler
            scheduler.add_job(
                job_func,
                trigger=trigger,
                id=view_name,
                name=view_name,
                replace_existing=True,
                executor=executor,
                misfire_grace_time=job_config.get('misfire_grace_time', 60),
                max_instances=job_config.get('max_instances', 3)
            )
            # TODO: Add print statement 

            util.logger.info(f"Scheduled job '{name}' for view '{view_name}' with actions {actions} to run {schedule_description}")
            util.SCHEDULED_JOBS[name] = {
                'actions': actions,
                'view_name': view_name,
                'schedule_description': schedule_description,
                'executor': executor,
                'enabled': True
            }
        except ValueError as e:
            util.logger.warning(f"Error scheduling job '{name}': {e}")
            continue

In [ ]:
#| export

@register('Create Job From View','Creates scheduled jobs for views that have job configuration.')
def create_job_from_view(data, context = None):
    """
    Create new jobs from views that have job configuration.
    """
    util.logger.info(f"Creating jobs from views...")
    util.logger.debug(f'Data: {data}')
    results = {}
    for view in data:
        try:
            util.logger.info(f"Processing view: {view}")
            view_data = get_view_data(view['TABLE_NAME'])
            if not view_data:
                util.logger.warning(f"No data returned from view: {view}")
                continue
            if 'CONFIG' not in view_data[0] and 'TK_CONFIG' not in view_data[0]:
                util.logger.warning(f"No job configuration found in view: {view}")
                continue

            first_row = view_data[0]
            util.logger.debug(f"Found job configuration in first row: {first_row}")
            config_col = 'CONFIG' if 'CONFIG' in first_row else 'TK_CONFIG'
            view_config = json.loads(first_row[config_col])

            util.logger.debug(f'Config data: {view_config}')
            name = view_config.get('name', view['TABLE_NAME'])
            view_name = view_config.get('view',view['TABLE_NAME'])
            actions = view_config.get('actions', [])
            enabled = view_config.get('enabled', True)
            executor = view_config.get('executor', 'default')                     
            
            if not enabled:
                util.logger.info(f"Job '{name}' is disabled, skipping")
                continue
                
            if not all([view_name, actions]) or not (view_config.get('schedule') or view_config.get('cron_schedule')):
                util.logger.info(f"Skipping invalid job configuration: {view}")
                util.logger.info(f"See debug for missing info.")
                util.logger.debug(f"Config Values:")
                util.logger.debug(f' - name: {name}')
                util.logger.debug(f' - view_name: {view_name}')
                util.logger.debug(f' - actions: {actions}')
                util.logger.debug(f' - enabled: {enabled}')
                continue
            
                # Check if job already exists with same configuration
            if name in util.SCHEDULED_JOBS:
                existing_job = util.SCHEDULED_JOBS[name]
                # Skip if job is identical
                if (existing_job['actions'] == actions and 
                    existing_job['view_name'] == view_name and 
                    existing_job['executor'] == executor):
                    util.logger.info(f"Job '{name}' already exists with same configuration, skipping")
                    results[view['TABLE_NAME']] = "Job already exists with same configuration"
                    continue

            job_func = create(view_name, actions, name, executor)
                
            try:
                # Create trigger from configuration
                trigger, schedule_description = _create_trigger_from_config(view_config)
                
                # Add job to scheduler
                scheduler.add_job(
                    job_func,
                    trigger=trigger,
                    id=view_name,
                    name=view_name,
                    replace_existing=True,
                    executor=executor,
                    misfire_grace_time=view_config.get('misfire_grace_time', 60),
                    max_instances=view_config.get('max_instances', 3)
                )
                
                util.logger.info(f"AUTO SCHEDULED job '{name}' based on view '{view_name}' with actions {actions} to run {schedule_description}")
                util.SCHEDULED_JOBS[name] = {
                    'actions': actions,
                    'view_name': view_name,
                    'schedule_description': schedule_description,
                    'executor': executor,
                    'enabled': True
                }
                results[view_name] = f"Successfully scheduled job '{name}'"
            except ValueError as e:
                util.logger.warning(f"Error scheduling job '{name}' from view '{view_name}': {e}")
                results[view_name] = f"Error scheduling job: {str(e)}"
        except Exception as e:
            util.logger.error(f"Error processing view '{view_name}': {str(e)}")
            results[view_name] = f"Error: {str(e)}"
    
    return results


2025-05-21 14:21:15,374 - triggerkit - INFO - Registered action: Create Job From View


In [ ]:
#| export

def run_scheduler():
    """
    Run the scheduler in the background.
    
    Returns:
        None
    """
    if scheduler.running:
        util.logger.warning("Scheduler is already running.")
        return
        
    util.logger.info("Starting APScheduler")
    scheduler.start()
    
    try:
        # Keep the main thread alive if this function is called directly
        while True:
            time.sleep(1)
    except (KeyboardInterrupt, SystemExit):
        util.logger.info("Shutting down scheduler")
        scheduler.shutdown()

# Additional utility functions for job management

def get_job_info(job_id: Optional[str] = None) -> Dict:
    """
    Get information about scheduled jobs.
    
    Args:
        job_id: Optional job ID to get specific job info
        
    Returns:
        Dictionary with job information
    """
    if job_id:
        job = scheduler.get_job(job_id)
        if job:
            next_run = job.next_run_time.strftime('%Y-%m-%d %H:%M:%S') if job.next_run_time else "Paused"
            return {
                'id': job.id,
                'next_run': next_run,
                'executor': job.executor,
                'config': util.SCHEDULED_JOBS.get(job.id, {})
            }
        return {'error': f"Job '{job_id}' not found"}
    
    jobs_info = {}
    for job in scheduler.get_jobs():
        next_run = job.next_run_time.strftime('%Y-%m-%d %H:%M:%S') if job.next_run_time else "Paused"
        jobs_info[job.id] = {
            'next_run': next_run,
            'executor': job.executor,
            'config': util.SCHEDULED_JOBS.get(job.id, {})
        }
    
    return jobs_info

def pause_job(job_id: str) -> Dict:
    """
    Pause a scheduled job.
    
    Args:
        job_id: Job ID to pause
        
    Returns:
        Dictionary with operation result
    """
    job = scheduler.get_job(job_id)
    if not job:
        return {'error': f"Job '{job_id}' not found"}
    
    scheduler.pause_job(job_id)
    
    if job_id in util.SCHEDULED_JOBS:
        util.SCHEDULED_JOBS[job_id]['enabled'] = False
    
    util.logger.info(f"Job '{job_id}' paused")
    return {'success': f"Job '{job_id}' paused"}

def resume_job(job_id: str) -> Dict:
    """
    Resume a paused job.
    
    Args:
        job_id: Job ID to resume
        
    Returns:
        Dictionary with operation result
    """
    job = scheduler.get_job(job_id)
    if not job:
        return {'error': f"Job '{job_id}' not found"}
    
    scheduler.resume_job(job_id)
    
    if job_id in util.SCHEDULED_JOBS:
        util.SCHEDULED_JOBS[job_id]['enabled'] = True
    
    util.logger.info(f"Job '{job_id}' resumed")
    return {'success': f"Job '{job_id}' resumed"}

def remove_job(job_id: str) -> Dict:
    """
    Remove a scheduled job.
    
    Args:
        job_id: Job ID to remove
        
    Returns:
        Dictionary with operation result
    """
    job = scheduler.get_job(job_id)
    if not job:
        return {'error': f"Job '{job_id}' not found"}
    
    scheduler.remove_job(job_id)
    
    if job_id in util.SCHEDULED_JOBS:
        del util.SCHEDULED_JOBS[job_id]
    
    util.logger.info(f"Job '{job_id}' removed")
    return {'success': f"Job '{job_id}' removed"}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()